In [1]:
import pandas as pd
import numpy as np
#from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.layers.core import Flatten
from keras.preprocessing.image import flip_axis
from keras.layers import Input, Lambda, Convolution2D, Dense, Dropout,Cropping2D
from keras.models import Model, model_from_json,Sequential
from scipy import misc
import random
import json
import csv
import os
import matplotlib.pyplot as plt
#import cv2
import time
%matplotlib inline

ImgDir=r'C:\Users\User\Desktop\CarND_BehaviorCloning\windows_sim\TrainingData\IMG\\'
LogFile=r'C:\Users\User\Desktop\CarND_BehaviorCloning\windows_sim/TrainingData/driving_log.csv'

UdacityDir=r'C:\Users\User\Desktop\CarND_BehaviorCloning\windows_sim\data\\'
UdacityLogFile=r'C:\Users\User\Desktop\CarND_BehaviorCloning\windows_sim\data\driving_log.csv'

print ("Hello")

Using TensorFlow backend.


Hello


In [2]:
def convertImage(image):
    
    cutImage=image#[60:140,:]
    
    line_image=np.copy(cutImage)
    
    hsv=cv2.cvtColor(line_image,cv2.COLOR_RGB2HSV)
    lower_yellow = np.array([23,41,133])
    upper_yellow = np.array([40,150,255])

    lower_red = np.array([150,0,0])
    upper_red = np.array([180,200,255])
    
    lower_black = np.array([0,0,0])
    upper_black = np.array([120,120,50])
    
    sensitivity=40
    lower_white = np.array([0,0,255-sensitivity], dtype=np.uint8)
    upper_white = np.array([255,sensitivity,255], dtype=np.uint8)
    
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    mask_red = cv2.inRange(hsv,lower_red,upper_red)
    mask_black = cv2.inRange(hsv, lower_black, upper_black)
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    
    masked_black = cv2.addWeighted(cutImage, 0.5,cv2.cvtColor(mask_black, cv2.COLOR_GRAY2RGB), 1, 0)
    masked_red= cv2.addWeighted(masked_black, 0.5,cv2.cvtColor(mask_red, cv2.COLOR_GRAY2RGB), 1, 0)
    masked_yellow= cv2.addWeighted(masked_red, 0.5,cv2.cvtColor(mask_yellow, cv2.COLOR_GRAY2RGB), 1, 0)
    masked_white= cv2.addWeighted(masked_yellow, 0.5,cv2.cvtColor(mask_white, cv2.COLOR_GRAY2RGB), 1, 0)
    
    mask= masked_white
    
    return mask

'def convertImage(image):\n    \n    cutImage=image#[60:140,:]\n    \n    line_image=np.copy(cutImage)\n    \n    hsv=cv2.cvtColor(line_image,cv2.COLOR_RGB2HSV)\n    lower_yellow = np.array([23,41,133])\n    upper_yellow = np.array([40,150,255])\n\n    lower_red = np.array([150,0,0])\n    upper_red = np.array([180,200,255])\n    \n    lower_black = np.array([0,0,0])\n    upper_black = np.array([120,120,50])\n    \n    sensitivity=40\n    lower_white = np.array([0,0,255-sensitivity], dtype=np.uint8)\n    upper_white = np.array([255,sensitivity,255], dtype=np.uint8)\n    \n    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)\n    mask_red = cv2.inRange(hsv,lower_red,upper_red)\n    mask_black = cv2.inRange(hsv, lower_black, upper_black)\n    mask_white = cv2.inRange(hsv, lower_white, upper_white)\n    \n    masked_black = cv2.addWeighted(cutImage, 0.5,cv2.cvtColor(mask_black, cv2.COLOR_GRAY2RGB), 1, 0)\n    masked_red= cv2.addWeighted(masked_black, 0.5,cv2.cvtColor(mask_red, cv

In [9]:
starttime=time.time()

lines=[]
measurements=[]
images=[]


with open(LogFile) as csvfile:
    reader=csv.reader(csvfile)
    for line in reader:
        lines.append(line)


for line in lines:
    for i in range (3):
        source_path=line[i]
        if i==0:
            correction=0
        if i==1:
            correction=0.2
        if i==2:
            correction=-0.2
        filename=source_path.split('\\')[-1]
        ##Dir=source_path.split('\\')[:-1]
        current_path=ImgDir+filename
        image=plt.imread(current_path)
        #image=convertImage(image)
        images.append(image)
        measurement=float(line[3])+correction
        measurements.append(measurement)
        if (measurement<-0.4 or measurement>0.4):
            flipImg=np.fliplr(image)
            images.append(flipImg)
            measurements.append(-measurement)


print ('Processing Time:', time.time()-starttime)
print (len(images),len(measurements))

Processing Time: 189.32282876968384
14423 14423


In [10]:
X_train=np.array(images)
y_train=np.array(measurements)

MemoryError: 

In [ ]:
#processingUdacityImages
lines2=[]
with open(UdacityLogFile) as csvfile:
    reader=csv.reader(csvfile)
    for line in reader:
        lines2.append(line)

for line in lines2[1:]:
    source_path=line[0]
    filename=source_path
    current_path=UdacityDir+filename
    image=plt.imread(current_path)
    #image=convertImage(image)
    images.append(image)
    measurement=float(line[3])
    measurements.append(measurement)
    flipImg=np.fliplr(image)
    images.append(flipImg)
    measurements.append(-measurement)
    
X_train=np.array(images)
y_train=np.array(measurements)

print (X_train.shape,y_train.shape)

In [11]:
model=Sequential()
model.add(Lambda(lambda x: x/255.-0.5,input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), strides=(2, 2), activation="relu")`
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), strides=(2, 2), activation="relu")`
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), strides=(2, 2), activation="relu")`
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`


In [12]:
model.compile(loss='mse',optimizer='adam')
model.fit(X_train,y_train,validation_split=0.3,shuffle=True,batch_size=128)
model.save('model.h5')

Train on 10096 samples, validate on 4327 samples
Epoch 1/10
10096/10096 [==============================] - 693s - loss: 0.0972 - val_loss: 0.1751
Epoch 2/10
10096/10096 [==============================] - 700s - loss: 0.0793 - val_loss: 0.1764
Epoch 3/10
10096/10096 [==============================] - 725s - loss: 0.0721 - val_loss: 0.1832
Epoch 4/10
10096/10096 [==============================] - 489s - loss: 0.0662 - val_loss: 0.1872
Epoch 5/10
10096/10096 [==============================] - 290s - loss: 0.0598 - val_loss: 0.1862
Epoch 6/10
10096/10096 [==============================] - 345s - loss: 0.0552 - val_loss: 0.1769
Epoch 7/10
10096/10096 [==============================] - 342s - loss: 0.0512 - val_loss: 0.1816
Epoch 8/10
10096/10096 [==============================] - 68361s - loss: 0.0471 - val_loss: 0.1780
Epoch 9/10
10096/10096 [==============================] - 1501s - loss: 0.0448 - val_loss: 0.1865
Epoch 10/10
10096/10096 [==============================] - 686s - loss: 0.0